In [ ]:
import numpy as np
import math
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import StaticAssets,QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import factset

import talib
import pandas as pd
import alphalens as al
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios


In [ ]:
universe = QTradableStocksUS

In [ ]:
def make_pipeline():
    asset_list = symbols(['SPY'])
    asset_filter = StaticAssets(asset_list)
    
    assets_moving_average = SimpleMovingAverage(inputs=[factset.Fundamentals.assets.latest], window_length=252)
    current_assets = factset.Fundamentals.assets.latest

    factor_to_analyze = (current_assets - assets_moving_average)

    return Pipeline(
        columns={'factor_to_analyze': factor_to_analyze},
        screen=asset_filter & factor_to_analyze.notnull()
    )

factor_data = run_pipeline(make_pipeline(), '2015-1-1', '2016-1-1')
pricing_data = get_pricing(factor_data.index.levels[1], '2015-1-1', '2016-6-1', fields='open_price')

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns


factor_data = get_clean_factor_and_forward_returns(
    result['percent_diff'],
    pricing,
    quantiles=2,
    periods=range(1,10,20) # Change the step to 10 or more for long look forward periods to save time
)
